# Basics of pySAS

There have been several updates to pySAS between versions 1.4 and 2.0. *Most* of the functionality is the same, but there are some differences.

## Initializing SAS

#### Starting pySAS on SciServer, Fornax, or Datalabs
SAS is automatically initialized when you start a container on either SciServer, Fornax, or Datalabs. You do **not** have to do anything. In a Jupyter Notebook or Python session you can import pySAS like normal:
```python
import pysas
```

#### Starting pySAS **for the first time** on a local machine

After installing `HEASOFT` and `SAS`, install pySAS using:
```
pip install xmmpysas
```
Then start a Python session and run
```python
import pysas
pysas.config_pysas.run_config()
```
This will step you through connecting pySAS to SAS. After SAS and pySAS have been configured pySAS will automatically initialize SAS when it is imported.
```python
import pysas
```

## Calling SAS Tasks

All standard SAS tasks are called using the `MyTask` object in pySAS.
```python
from pysas.sastask import MyTask
```
`MyTask` takes two inputs, the first is the name of the SAS task as a string. The second is a dictionary with the input arguments for the SAS task. If there are no input arguments then the dictionary will be empty:
```python
MyTask('sasversion', {}).run()
```

Each input argument will take the form of a 'key' (parameter name) and 'value' (parameter value) pair. Each parameter value should be a single string.
```python
inargs = {'table'           : 'event_list.fits',
          'withfilteredset' : 'yes',
          "expression"      : "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'",
          'filteredset'     : 'filtered_event_list.fits',
          'filtertype'      : 'expression', 
          'keepfilteroutput': 'yes',
          'updateexposure'  : 'yes',
          'filterexposure'  : 'yes'}

MyTask('evselect', inargs).run()
```
<div class="alert alert-block alert-warning">
    <b>Note:</b> Some inputs require single quotes to be preserved in the string. This can be done using double quotes to form the string. i.e. "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'"
</div>

If you need to pass in command line options to a SAS task, add it to the dictionary with they key '`options`'. For example,
```python
inargs = {'options'         : '--verbosity 6 --noclobber'
          'table'           : 'event_list.fits',
          'withfilteredset' : 'yes',
          "expression"      : "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'",
          'filteredset'     : 'filtered_event_list.fits',
          'filtertype'      : 'expression', 
          'keepfilteroutput': 'yes',
          'updateexposure'  : 'yes',
          'filterexposure'  : 'yes'}

MyTask('evselect', inargs).run()
```

## Downloading Observation Data Files

To work with a particular observation in a Python session the user types:
```python
obsid = '##########'
my_obs = pysas.obsid.ObsID(obsid)
```
inserting the desired obsID. Then run the function `basic_setup`. The function `basic_setup` will download the `ODF` data, run `calibrate_odf` and then run `epproc`, `emproc`, and `rgsproc` with the option of running `epchain` instead of `epproc` and `emchain` instead of `emproc`. 
```python
my_obs.basic_setup()
```
If the ODF files have previously been downloaded and are in the default data directory, then upon creating the ObsID object (i.e. "my_obs = pysas.obsid.ObsID(obsid)") pySAS will automatically find and link all important summary and calibration files, and also all previously generated event lists made by `epproc`, `emproc`, and `rgsproc`.

### Downloading Proprietary Data using the Development Version of pySAS

Proprietary data can be downloaded from both the XSA and the HEASARC. How each handles proprietary data is slightly different. For the XSA proprietary data requires your Cosmos username and password. To download proprietary data from the XSA using pySAS the user creates a ODF object like normal,

```python
obsid = '##########'
my_obs = pysas.obsid.ObsID(obsid)
```
Then when you run either `basic_setup` just indicate that the Obs ID is proprietary.

```python
my_obs.basic_setup(proprietary=True)
```

The data request is made using astroquery and astroquery will ask for the user's Cosmos username and password. The [ESA module for astroquery](https://github.com/astropy/astroquery/blob/main/astroquery/esa/xmm_newton/core.py) allows many more inputs for specifying exactly what XMM data to download. All of those inputs can be passed in as additional arguments to `basic_setup`. Those inputs will be passed on to astroquery.

Proprietary data from the HEASARC comes encrypted. Anyone can download it, but you need an encryption key to unpack the data. The encryption key is a alphanumeric string with 30 characters. You provide the encryption key to `basic_setup` and pySAS will handle the rest.

```python
my_obs.basic_setup(repo='heasarc',encryption_key='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
```

### Default Configuration

The default data directory can be set or changed later using the function `set_sas_config_option()`.

For example:
```python
from pysas.configutils import set_sas_config_option
data_path = '/path/to/new/data/dir/'
set_sas_config_option('data_dir', data_path)
```

The default values for the SAS directory (`sas_dir`), the path to the calibration files (`sas_ccfpath`), along with `verbosity` and `suppress_warning`, can also be set in the same way.

At any time the user can clear all previous values with,
```python
from pysas.configutils import clear_sas_defaults
clear_sas_defaults()
```
pySAS uses the standard Python configuration module [configparser](https://docs.python.org/3/library/configparser.html).

The configuration file for SAS can be found in the user's `HOME` directory (or the `XDG_CONFIG_HOME` directory which is usually the user's `HOME` directory) inside the hidden directory `.config`, then inside the `sas` directory. The configuration file will be named `sas.cfg`. It can be opened and edited by any text editor.

The typical contents of the file might be:
```
[DEFAULT]
sas_dir = /does/not/exist
sas_ccfpath = /does/not/exist
data_dir = /does/not/exist
verbosity = 4
pysas_verbosity = WARNING
suppress_warning = 1
on_sci_server = False

[sas]
data_dir = /home/rtanner/xmm_data
sas_dir = /home/rtanner/sas/xmmsas_22.1.0-a8f2c2afa-20250304
sas_ccfpath = /home/rtanner/sas/CCF
verbosity = 2
pysas_verbosity = INFO
```
The values in `[sas]` will override the `[DEFAULT]` values. These values can be edited by hand, but it is *strongly* recommended to use the function included with pySAS to change the default values:
```python
from pysas.configutils import set_sas_config_option
set_sas_config_option('option_to_set','value_for_option')
```

---
### ObsID

`ObsID` is a new pySAS module that makes it easier for the user to interface with pySAS. It contains useful functions along with links to important directories and files.

When a user creates an `ObsID` object using:
```python
obsid = '##########'
my_obs = pysas.obsid.ObsID(obsid)
```
The `ObsID` object will have a few important links and information by default. These include:

    - obsid    : The Obs ID number used to create the ObsID object.
    - data_dir : Path to base data directory where all XMM data files 
                 will be downloaded. (/path/to/data_dir)
    - obs_dir  : Path to directory containing the data files 
                 (obs_dir --> /path/to/data_dir/obsid).
    - odf_dir  : Path to directory with the raw observation data files 
                 (odf_dir --> obs_dir/ODF --> data_dir/obsid/ODF).
    - work_dir : The working directory when SAS tasks will be run 
                 (work_dir --> obs_dir/work --> data_dir/obsid/work).
    - sas_ccf  : Link to the 'ccf.cif' file, if it exists. By default the 
                 'ccf.cif' file will be created in the work_dir.
    - sas_odf  : Link to the '*SUM.SAS' file, if it exists. By default the 
                 '*SUM.SAS' file will be created in the work_dir.
    - If they exist, links to the basic event lists will be stored in a 
      dictionary named 'files'.
      
      The 'files' dictionary will have the following keys:
      
            'sas_ccf'
            'sas_odf'
            'PNevt_list'
            'M1evt_list'
            'M2evt_list'
            'R1evt_list'
            'R2evt_list'
            'OMevt_list'

There is an entire document explaing the features that can be fo [Using pySAS v2](./Using_pySASv2.ipynb).pynb).

---
### Input Arguments

As noted above, for all SAS tasks, input arguments should be in a dictionary, with each parameter value a single string. 

For example,
```python
inargs = {'table'            : 'event_list.fits', 
          'withfilteredset'  : 'yes', 
          'expression'       : "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'", 
          'filteredset'      : 'filtered_event_list.fits', 
          'filtertype'       : 'expression', 
          'keepfilteroutput' : 'yes', 
          'updateexposure'   : 'yes', 
          'filterexposure'   : 'yes'}

MyTask('evselect', inargs).run()
```
or it could even be written like this:

```python
inargs = {}
inargs['table']            = 'event_list.fits'
inargs['withfilteredset']  = 'yes'
inargs['expression']       = "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'"
inargs['filteredset']      = 'filtered_event_list.fits'
inargs['filtertype']       = 'expression'
inargs['keepfilteroutput'] = 'yes'
inargs['updateexposure']   = 'yes'
inargs['filterexposure']   = 'yes'

MyTask('evselect', inargs).run()
```

Then if a single input needs to be changed and the same SAS task run again the user could simply use:

```python
inargs['expression'] = "'(PATTERN <= 12)&&(PI in [4000:12000])&&#XMMEA_EM'"
MyTask('evselect', inargs).run()
```

For versions of pySAS before v2.0, the input arguments were passed in as a list. And in pySAS v2.0 you can still pass in the input arguments as a list. pySAS v2.0 is fully backwards compatible.
```python
inargs = ['table=event_list.fits', 
          'withfilteredset=yes', 
          "expression='(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'", 
          'filteredset=filtered_event_list.fits', 
          'filtertype=expression', 
          'keepfilteroutput=yes', 
          'updateexposure=yes', 
          'filterexposure=yes']

MyTask('evselect', inargs).run()
```

While the values for all input arguments should be in string format, pySAS v2.0 will accept numbers, as shown below. These numbers will be converted into strings automatically.

```python
inargs = {}
inargs['spectrumset'] = 'R1_spectra.fits'
inargs['rmfset']      = 'rmf1_file.fits'
inargs['evlist']      = 'R1_event_list'
inargs['emin']        = 0.4
inargs['emax']        = 2.5
inargs['rows']        = 4000

MyTask('rgsrmfgen', inargs).run()
```

Also in pySAS v2.0 'yes/no' parameters can be passed in as a Python boolean:
```python
inargs = {'table'            : 'event_list.fits', 
          'withfilteredset'  : True, 
          'expression'       : "'(PATTERN <= 12)&&(PI in [200:4000])&&#XMMEA_EM'", 
          'filteredset'      : 'filtered_event_list.fits', 
          'filtertype'       : 'expression', 
          'keepfilteroutput' : True, 
          'updateexposure'   : True, 
          'filterexposure'   : True}

MyTask('evselect', inargs).run()
```

The user can get all the default parameters for any SAS task by using:
```python
task_name = 'insert_task_name'
inargs = pysas.param.get_input_params(task_name)
```

This will return a special obejct that behaves just like a normal Python dictionary with all possible input arguments for a given SAS task, along with their default values. The user can modify the values in this dictionary and pass it back in to `MyTask`.

<div class="alert alert-block alert-warning">
    <b>Note:</b> This is still experimental. We have tested this, but it is possible to run into some unexpected behavior. We are aware of at least one (!) case with unexpected behavior for an uncommonly used SAS task.
</div>